In [2]:

# 导入数据
flux_data <- read.csv("/kaggle/input/aaa/flux.csv")

# 合并日期
flux_data$日期 <- as.Date(paste(flux_data$year, flux_data$month, flux_data$day), format = "%Y %m %d")

# 筛选数据
selected_site_data <- subset(flux_data, site == 'IT-Lav')

# 输出数据
write.table(selected_site_data, file = paste0("学号", "姓名", ".txt"), row.names = FALSE)

# 查看前20行
head(selected_site_data, 20)


Warning message in file(file, "rt"):
“cannot open file '/kaggle/input/aaa/flux.csv': No such file or directory”


ERROR: Error in file(file, "rt"): cannot open the connection


In [ ]:
# 处理缺失值，将-9999或者空格替换为NA
selected_site_data[selected_site_data == -9999] <- NA
selected_site_data[selected_site_data == ""] <- NA

# 求平均
site_monthly_avg <- aggregate(selected_site_data[, c("nee", "reco", "gpp")], 
                              by = list(site = selected_site_data$site, 
                                        month = format(selected_site_data$日期, "%Y-%m")), 
                              FUN = mean)

# 描述统计
summary_stats <- lapply(site_monthly_avg[, -c(1, 2)], summary)
View(summary_stats)


In [ ]:
# 加载ggplot2包
if (!require(ggplot2)) install.packages("ggplot2")
library(ggplot2)

# 绘制季节变化图（nee），reco,gpp直接c+v
ggplot(data = selected_site_data, aes(x = 日期, y = nee, color = site)) +
  geom_line() +
  facet_wrap(~ site) +
  labs(x = "日期", y = "NEE (µmolCO2 /m2/s)", title = "NEE季节变化图")



In [ ]:
if (!require(car)) install.packages("car")
library(car)

if (!require(MASS)) install.packages("MASS")
library(MASS)

In [ ]:
independent_vars <- c("Ta", "VPD", "Pa", "ws", "rain", "co2", "swc", "et")
dependent_vars <- c("nee", "reco", "gpp")

# VIF检验
for (dep_var in dependent_vars) {
  model_vif <- vif(lm(paste(dep_var, "~ .", collapse = "+", sep = ""), data = selected_site_data[independent_vars]))
  print(paste("VIF for", dep_var, ":"))
  print(model_vif)
}

In [ ]:
# nee进行逐步回归分析(reco,gpp)
full_model <- lm(NEE ~ ., data = selected_site_data[independent_vars + "nee"])
step_model <- stepAIC(full_model, direction = "both", trace = FALSE)

summary(step_model)

In [ ]:
if (!require(agricolae)) install.packages("agricolae")
library(agricolae)

In [ ]:
independent_vars <- c("Ta", "VPD", "Pa", "ws", "rain", "co2", "swc", "et")

In [ ]:
# my_version <- R.Version()   
# my_version

In [ ]:
# nee,gpp,reco
path_analysis_data <- selected_site_data[, c("nee", independent_vars)]
# 通径分析
path.model <- pathtest(path_analysis_data)
summary(path.model)